In [1]:
import pandas as pd
import numpy as np
import ipynb.fs.full.utils as utils
import importlib
importlib.reload(utils)
from keras.models import Sequential, Model, load_model
from keras.layers.core import Dense, Dropout, Reshape, Activation, Lambda, Flatten
from keras.layers import TimeDistributed, BatchNormalization, Input, Conv1D
from keras.layers.merge import dot
from keras.optimizers import Adam
from keras.callbacks import TensorBoard
import keras
import matplotlib.pyplot as pltё
import seaborn as sns
import lightgbm as lgb
from sklearn import preprocessing, metrics
import pyspark.sql.functions as SF

Using TensorFlow backend.
/home/timothyxp/yes/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/timothyxp/yes/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/timothyxp/yes/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/timothyxp/yes/lib/python3.7/site-packages/tensorflow/python

In [2]:
sales = pd.read_csv("sales_train_validation.csv")
sales.head(1)

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,3,0,1,1,1,3,0,1,1


In [57]:
prices = pd.read_csv("sell_prices.csv")
prices.head(1)

,store_id,item_id,wm_yr_wk,sell_price
0,CA_1,HOBBIES_1_001,11325,9.58


In [58]:
calendar = pd.read_csv("calendar.csv")
calendar.head(1)

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0


In [59]:
prices = prices.merge(calendar[["wm_yr_wk", "d"]], on="wm_yr_wk", how="left")

In [60]:
prices.shape

(47735397, 5)

In [61]:
prices.head()

,store_id,item_id,wm_yr_wk,sell_price,d
0,CA_1,HOBBIES_1_001,11325,9.58,d_897
1,CA_1,HOBBIES_1_001,11325,9.58,d_898
2,CA_1,HOBBIES_1_001,11325,9.58,d_899
3,CA_1,HOBBIES_1_001,11325,9.58,d_900
4,CA_1,HOBBIES_1_001,11325,9.58,d_901


In [62]:
prices["d"] = prices["d"].apply(lambda x: int(x[2:]))

In [63]:
prices.d.max()

1969

In [64]:
prices.drop(columns=["wm_yr_wk"], inplace=True)

In [65]:
prices.head()

,store_id,item_id,sell_price,d
0,CA_1,HOBBIES_1_001,9.58,897
1,CA_1,HOBBIES_1_001,9.58,898
2,CA_1,HOBBIES_1_001,9.58,899
3,CA_1,HOBBIES_1_001,9.58,900
4,CA_1,HOBBIES_1_001,9.58,901


In [66]:
new_df= []
for (store_id, item_id), group in prices.groupby(["store_id", "item_id"]):
    prices_list = []
    group = group.sort_values("d")
    
    price = group["sell_price"].tolist()
    days = group["d"].tolist()
    j = 0
    
    for i in range(1, 1970):
        if days[j] == i:
            prices_list.append(price[j])
            j+=1
        else:
            prices_list.append(0)
        
    
    new_df.append([store_id, item_id] + prices_list)
    
    
# because of pandas bug in pivot

In [69]:
new_prices = pd.DataFrame(new_df, columns=["store_id", "item_id"] + [f"sell_price_{i}" for i in range(1, 1970)])

In [70]:
new_prices.head()

,store_id,item_id,sell_price_1,sell_price_2,sell_price_3,sell_price_4,sell_price_5,sell_price_6,sell_price_7,sell_price_8,...,sell_price_1960,sell_price_1961,sell_price_1962,sell_price_1963,sell_price_1964,sell_price_1965,sell_price_1966,sell_price_1967,sell_price_1968,sell_price_1969
0,CA_1,FOODS_1_001,2.00,2.00,2.00,2.00,2.00,2.00,2.00,2.00,...,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24
1,CA_1,FOODS_1_002,7.88,7.88,7.88,7.88,7.88,7.88,7.88,7.88,...,9.48,9.48,9.48,9.48,9.48,9.48,9.48,9.48,9.48,9.48
2,CA_1,FOODS_1_003,2.88,2.88,2.88,2.88,2.88,2.88,2.88,2.88,...,3.23,3.23,3.23,3.23,3.23,3.23,3.23,3.23,3.23,3.23
3,CA_1,FOODS_1_004,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,1.96,1.96,1.96,1.96,1.96,1.96,1.96,1.96,1.96,1.96
4,CA_1,FOODS_1_005,2.94,2.94,2.94,2.94,2.94,2.94,2.94,2.94,...,3.54,3.54,3.54,3.54,3.54,3.54,3.54,3.54,3.54,3.54


In [72]:
sales = sales.merge(new_prices, on=["store_id", "item_id"], how="left")

In [73]:
sales.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,sell_price_1960,sell_price_1961,sell_price_1962,sell_price_1963,sell_price_1964,sell_price_1965,sell_price_1966,sell_price_1967,sell_price_1968,sell_price_1969
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,8.38,8.38,8.38,8.38,8.38,8.38,8.38,8.38,8.38,8.38
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,3.97,3.97,3.97,3.97,3.97,3.97,3.97,3.97,3.97,3.97
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2.97,2.97,2.97,2.97,2.97,2.97,2.97,2.97,2.97,2.97
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,4.64,4.64,4.64,4.64,4.64,4.64,4.64,4.64,4.64,4.64
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2.88,2.88,2.88,2.88,2.88,2.88,2.88,2.88,2.88,2.88


In [140]:
def get_target_columns(i):
    return [f"d_{i}" for i in range(i + 1, i + 29)]

def get_feature_columns(i):
    return [f"d_{i}" for i in range(i - 1120 + 1, i + 1)] + [f"sell_price_{i}" for i in range(i - 1120 + 1, i + 28 + 1)]

In [153]:
test_columns = get_feature_columns(1913 - 28*2)
test_target_columns = get_target_columns(1913 - 28*2)

val_columns = get_feature_columns(1913 - 28)
val_target_columns = get_target_columns(1913 - 28)

In [248]:
def get_model(activation = "relu"):
    model = Sequential()
    model.add(Dense(256, input_shape=(train_data.shape[1],)))
    model.add(BatchNormalization())
    model.add(Activation(activation))
    model.add(Dropout(0.4))
    model.add(Dense(128))
    model.add(BatchNormalization())
    model.add(Activation(activation))
    model.add(Dense(64))
    model.add(BatchNormalization())
    model.add(Activation(activation))
    model.add(Dense(32))
    model.add(BatchNormalization())
    model.add(Activation(activation))
    model.add(Dropout(0.4))
    model.add(Dense(28)) # result layer
    return model

In [250]:
train_data = []
target = []

for k in range(3, 15):
    train_columns = get_feature_columns(1913-28*k)
    target_columns = get_target_columns(1913-28*k)
    columns_renaming = dict([(column, f"feature_{i}") for i, column in enumerate(train_columns)])
    target_columns_renaming = dict([(column, f"target_{i}") for i, column in enumerate(target_columns)])
    
    train_data.append(sales[train_columns].fillna(0).rename(columns = columns_renaming))
    target.append(sales[target_columns].fillna(0).rename(columns = target_columns_renaming))

train_data = pd.concat(train_data).values
target = pd.concat(target).values
print(train_data.shape)
    
test_data = sales[test_columns].fillna(0).values
test_target = sales[test_target_columns].fillna(0).values
print(test_data.shape)

val_data = sales[val_columns].fillna(0).values
val_target = sales[val_target_columns].fillna(0).values

model = get_model()

lr = 0.0002
decay = 2e-6

model.compile(
    loss="mse",
    optimizer=Adam(lr, decay=decay),
    metrics=["mae"]
)

tensorboard_callback = TensorBoard(
    log_dir=f"/home/timothyxp/kaggle/m5_accuracy/tensorboard_logs/mlp_simple_fe_more_examples_lr={lr}_decay={decay}_ex=2")
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save = ModelCheckpoint(f'mlp.bin', save_best_only=True, monitor='val_loss', mode='min')
#reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=7, verbose=1, epsilon=1e-4, mode='min')

model.fit(
    train_data,
    target,
    batch_size=512,
    epochs=200,
    verbose=1,
    shuffle=True,
    callbacks=[tensorboard_callback, earlyStopping, mcp_save],
    validation_data=(
        test_data,
        test_target
        )
)

predictions = model.predict(val_data)
print(f"val score - {mean_squared_error(val_target, predictions)}")    

(365880, 2268)
(30490, 2268)
Train on 365880 samples, validate on 30490 samples
Epoch 1/200
365880/365880 [==============================] - 43s 119us/step - loss: 9.2537 - mae: 1.1932 - val_loss: 5.7428 - val_mae: 1.0346
Epoch 2/200
365880/365880 [==============================] - 40s 109us/step - loss: 6.3454 - mae: 1.0813 - val_loss: 5.2345 - val_mae: 1.1590
Epoch 3/200
365880/365880 [==============================] - 40s 109us/step - loss: 5.9011 - mae: 1.0821 - val_loss: 5.1329 - val_mae: 1.0656
Epoch 4/200
365880/365880 [==============================] - 40s 108us/step - loss: 5.7176 - mae: 1.0737 - val_loss: 5.1103 - val_mae: 1.0995
Epoch 5/200
365880/365880 [==============================] - 40s 109us/step - loss: 5.5864 - mae: 1.0679 - val_loss: 5.1938 - val_mae: 1.0755
Epoch 6/200
365880/365880 [==============================] - 40s 108us/step - loss: 5.4704 - mae: 1.0611 - val_loss: 5.0685 - val_mae: 1.0597
Epoch 7/200
365880/365880 [==============================] - 40s 109

In [251]:
inference_columns = get_feature_columns(1913)

In [254]:
utils.save_pickle("mlp_last.bin", model)

In [255]:
from keras.models import load_model

prediction_columns = [f"F{i}" for i in range(1, 29)]

model = load_model(f'mlp.bin')
inference_data = sales[inference_columns].fillna(0).values
predictions = model.predict(inference_data)
submission = sales[["id"]].reset_index(drop=True)
preds = pd.DataFrame(predictions, columns=prediction_columns)
for column in prediction_columns:
    submission[column] = preds[column]

(3049, 29)


In [256]:
submission.shape

(30490, 29)

In [257]:
sample = pd.read_csv('sample_submission.csv')

In [258]:
sample = sample[~sample.id.isin(submission.id)]

In [259]:
submission = pd.concat([submission, sample])
submission.shape

(60980, 29)

In [260]:
submission.to_csv("mlp_prediction.csv", index=False)

In [185]:
submission

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_validation,0.340615,0.262910,0.384804,0.341908,0.505944,0.557778,0.573432,0.236152,0.402271,...,0.467142,0.510237,0.573861,0.412129,0.382783,0.320374,0.438118,0.319466,0.440226,0.544641
1,HOBBIES_1_002_CA_1_validation,0.152780,0.167198,0.206063,0.135991,0.220261,0.292488,0.271231,0.096720,0.238832,...,0.259139,0.246276,0.268840,0.222911,0.179055,0.157440,0.205026,0.173557,0.235826,0.277211
2,HOBBIES_1_003_CA_1_validation,0.225051,0.255311,0.302572,0.278527,0.349166,0.458023,0.398659,0.214990,0.313159,...,0.336577,0.425645,0.346138,0.328538,0.210052,0.305143,0.245748,0.245530,0.400174,0.444386
3,HOBBIES_1_004_CA_1_validation,0.528042,0.909657,1.076101,1.073094,1.354179,2.175359,1.822797,0.734706,1.178146,...,1.427944,1.816013,1.350976,1.155424,1.111345,1.417467,1.050518,1.200001,1.635054,1.958140
4,HOBBIES_1_005_CA_1_validation,0.404196,0.403201,0.541060,0.443906,0.516849,0.801108,0.749415,0.420884,0.492820,...,0.582676,0.744279,0.513144,0.486561,0.419561,0.478995,0.397129,0.460296,0.614950,0.699145
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60975,FOODS_3_823_WI_3_evaluation,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
60976,FOODS_3_824_WI_3_evaluation,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
60977,FOODS_3_825_WI_3_evaluation,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
60978,FOODS_3_826_WI_3_evaluation,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [264]:
melt_submission = submission[:30490].melt(id_vars=["id"], var_name="day", value_name="demand")

In [267]:
melt_submission["day"] = melt_submission["day"].apply(lambda s: int(s[1:]) + 1913)
melt_submission.head(2)

,id,day,demand
0,HOBBIES_1_001_CA_1_validation,1914,0.519515
1,HOBBIES_1_002_CA_1_validation,1914,0.252443


In [268]:
melt_submission.to_csv("melt_mlp_predictions.csv", index=False)